In [1]:
FILENAME = '1'

import time
import pickle
import numpy as np

# Create socket to send read data in a consistent way at the corresponding times
from Raspi.client import get, send

with open(FILENAME + '.pkl', 'rb') as handle:
    out, waypoints = pickle.load(handle)

if len(out) == 0:
    raise SystemExit("Empty File")



TimeOffset = time.time()-min(out[0][0][-1], waypoints[0][-1])+3
send(TimeOffset)
out = [[i[:-1]+[i[-1]+TimeOffset] for i in j] for j in out]
LastTime = out[-1][-1]

buffer = False
flag = False
while len(out):
    while len(out) and time.time() > out[0][0][-1]:
        buffer = out[0]
        out = out[1:]
    
    if flag != ["Get Lidar"]: flag = get(blocking=False)
    if flag == ["Get Lidar"] and buffer != False:
        send(buffer)
        buffer = False
        flag = False